In [1]:
from semantic_caching import SemanticCaching

In [2]:
cache = SemanticCaching()

[INFO] Initializing mongodb.....  complete (0.1956s)


In [3]:
cache.init_vector_db()

[INFO] Initializing encoder... complete (2.4045s)
[INFO] Initializing Qdrant vector db.....  complete (0.5873s)


In [4]:
question = 'Which one is the first country created in the world?'
cache.ask(question, use_llm=False)

CUDA is not available for this device. Defaulting to ARES API.
[INFO] Searching for the answer in Qdrant. complete (0.0651s)
[INFO] Retrieving from cache ...  complete (0.0004s)


'The first country created in the world is Egypt.'

In [6]:
question = 'How far is the Earth from Mars?'
cache.ask(question, use_llm=True)

CUDA is not available for this device. Defaulting to ARES API.
[INFO] Searching for the answer in Qdrant. complete (0.1345s)
[INFO] Retrieving from cache ...  complete (0.0004s)


"The distance from Earth to Mars can vary depending on the positions of the two planets in their orbits. On average, the distance is about 140 million miles (225 million km) from Earth to Mars. However, the distance can range from 33.9 million miles (54.6 million km) to 250 million miles (401 million km). The minimum distance between Earth and Mars is about 33.9 million miles (54.6 million kilometers), while the maximum distance can be up to 250 million miles (401 million kilometers). \n\nSources:\n- Study.com: [Distance from Earth to Mars | Facts & Measurement](https://study.com/academy/lesson/mars-distance-earth-fact-measurement.html)\n- NASA: [Hazard: Distance From Earth](https://www.nasa.gov/hrp/hazard-distance-from-earth/)\n- Space.com: [Distance to Mars: How far away is the Red Planet?](https://www.space.com/16875-how-far-away-is-mars.html)\n- TheSkyLive: [How Far Away Is Mars?](https://theskylive.com/how-far-is-mars)\n- Mars.nasa.gov: [Mars Close Approach | Mars in our Night Sky

## Test with custom RAG architecture

In [ ]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-mpnet-base-v2')

In [ ]:
question = 'who are the speakers'
encoding = encoder.encode(question)

In [ ]:
# Pinecone
from pinecone import Pinecone
import os
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])

In [ ]:
index_name = 'webinar-transcript'
index = pc.Index(index_name)

res = index.query(vector=encoding, top_k=10)
print(res)